In [116]:
# Import packages
import pandas as pd
import numpy as np

# Read CSV from path
path = "data/mapdataall.csv"

fire_data = pd.read_csv(path)

In [117]:
# Select and rename columns from original csv data
fire_data_columns = ["incident_name", "incident_id", "incident_county", 
                     "incident_longitude", "incident_latitude", "incident_acres_burned", 
                     "incident_dateonly_created", "incident_dateonly_extinguished"]

fire_columns_rename = fire_data[fire_data_columns].rename(columns={
    "incident_name":"Name of Fire",
    "incident_id":"Fire ID",
    "incident_county": "County", 
    "incident_longitude": "Longitude", 
    "incident_latitude": "Latitude", 
    "incident_acres_burned": "Acres Burned", 
    "incident_dateonly_created": "Date Created", 
    "incident_dateonly_extinguished": "Date Extinguished"})

In [118]:
# # Select and rename columns from original csv data
# fire_data_columns = ["incident_name", "incident_id", "incident_county", 
#                      "incident_longitude", "incident_latitude", "incident_acres_burned", 
#                      "incident_date_created", "incident_date_extinguished"]

# fire_columns_rename = fire_data[fire_data_columns].rename(columns={
#     "incident_name":"Name of Fire",
#     "incident_id":"Fire ID",
#     "incident_county": "County", 
#     "incident_longitude": "Longitude", 
#     "incident_latitude": "Latitude", 
#     "incident_acres_burned": "Acres Burned", 
#     "incident_date_created": "Date Created", 
#     "incident_date_extinguished": "Date Extinguished"})

In [119]:
#Clean and only look at Central California data
fire_data_clean = fire_columns_rename.dropna()

central_county = ["Sierra", "Nevada", "Placer", "Yuba", "Sacramento", "El Dorado", "Amador", "Alpine", "Calaveras", "Butte"]
central_county_data = fire_data_clean[fire_data_clean["County"].isin(central_county)]

central_county_data

,Name of Fire,Fire ID,County,Longitude,Latitude,Acres Burned,Date Created,Date Extinguished
31,Lone Fire,1a5192f1-bc99-4130-8683-381e0534b76a,Butte,-121.576804,39.434183,298.0,2013-06-03,2013-06-03
40,Roadrunner Fire,bf59bc5b-1398-4ae5-adef-6f61b9ee1b90,Amador,-120.966016,38.277830,96.0,2013-06-12,2013-06-13
44,PFE Fire,fe47b1c2-b9c0-44be-8d10-f270056db35c,Placer,-121.380862,38.729297,237.0,2013-06-13,2013-06-13
63,McGanney Fire,397ed8e6-6088-4e03-9316-8f50c65a83be,Yuba,-121.316340,39.206000,73.0,2013-07-08,2013-07-08
65,Kyburz Fire,d778a3f0-2d23-43c1-b9bc-1ea33898fab2,El Dorado,-120.298000,38.774000,572.0,2013-07-08,2013-07-11
...,...,...,...,...,...,...,...,...
2002,Apple Fire,c7cb91c5-4423-4965-8f6d-ad9c711a376d,Yuba,-121.304167,39.323167,39.0,2022-07-27,2022-08-02
2019,Oak Fire,ad465a00-5520-4188-9f91-3c5568adb40e,Placer,-120.952600,39.049410,22.0,2022-08-15,2022-08-31
2024,Pleasant Fire,8d4e0591-dd62-4c04-82f3-10eb76556cde,Nevada,-121.132900,39.280500,48.0,2022-08-20,2022-08-29
2028,Still Fire,4612ddda-90b6-4733-a0a5-6bdeb622614b,Nevada,-121.110300,39.054300,44.0,2022-08-27,2022-08-29


In [122]:
# Change date columns to datetime data type
central_county_data["Date Created"] = pd.to_datetime(central_county_data["Date Created"])
central_county_data["Date Extinguished"] = pd.to_datetime(central_county_data["Date Extinguished"])
central_county_data["Year"] = central_county_data["Date Created"].dt.year

# Create column for duration of fire
central_county_data["Duration (Days)"] = (central_county_data["Date Extinguished"] - central_county_data["Date Created"]).dt.days
clean_central_data = central_county_data[central_county_data["Duration (Days)"]>=0]

clean_central_data

c:\Users\Estelle\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\Estelle\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Users\Estelle\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

,Name of Fire,Fire ID,County,Longitude,Latitude,Acres Burned,Date Created,Date Extinguished,Year,Duration (Days)
31,Lone Fire,1a5192f1-bc99-4130-8683-381e0534b76a,Butte,-121.576804,39.434183,298.0,2013-06-03,2013-06-03,2013,0
40,Roadrunner Fire,bf59bc5b-1398-4ae5-adef-6f61b9ee1b90,Amador,-120.966016,38.277830,96.0,2013-06-12,2013-06-13,2013,1
44,PFE Fire,fe47b1c2-b9c0-44be-8d10-f270056db35c,Placer,-121.380862,38.729297,237.0,2013-06-13,2013-06-13,2013,0
63,McGanney Fire,397ed8e6-6088-4e03-9316-8f50c65a83be,Yuba,-121.316340,39.206000,73.0,2013-07-08,2013-07-08,2013,0
65,Kyburz Fire,d778a3f0-2d23-43c1-b9bc-1ea33898fab2,El Dorado,-120.298000,38.774000,572.0,2013-07-08,2013-07-11,2013,3
...,...,...,...,...,...,...,...,...,...,...
2002,Apple Fire,c7cb91c5-4423-4965-8f6d-ad9c711a376d,Yuba,-121.304167,39.323167,39.0,2022-07-27,2022-08-02,2022,6
2019,Oak Fire,ad465a00-5520-4188-9f91-3c5568adb40e,Placer,-120.952600,39.049410,22.0,2022-08-15,2022-08-31,2022,16
2024,Pleasant Fire,8d4e0591-dd62-4c04-82f3-10eb76556cde,Nevada,-121.132900,39.280500,48.0,2022-08-20,2022-08-29,2022,9
2028,Still Fire,4612ddda-90b6-4733-a0a5-6bdeb622614b,Nevada,-121.110300,39.054300,44.0,2022-08-27,2022-08-29,2022,2


In [125]:
# Reset the index
clean_central_data = clean_central_data.reset_index(drop=True)
clean_central_data

,Name of Fire,Fire ID,County,Longitude,Latitude,Acres Burned,Date Created,Date Extinguished,Year,Duration (Days)
0,Lone Fire,1a5192f1-bc99-4130-8683-381e0534b76a,Butte,-121.576804,39.434183,298.0,2013-06-03,2013-06-03,2013,0
1,Roadrunner Fire,bf59bc5b-1398-4ae5-adef-6f61b9ee1b90,Amador,-120.966016,38.277830,96.0,2013-06-12,2013-06-13,2013,1
2,PFE Fire,fe47b1c2-b9c0-44be-8d10-f270056db35c,Placer,-121.380862,38.729297,237.0,2013-06-13,2013-06-13,2013,0
3,McGanney Fire,397ed8e6-6088-4e03-9316-8f50c65a83be,Yuba,-121.316340,39.206000,73.0,2013-07-08,2013-07-08,2013,0
4,Kyburz Fire,d778a3f0-2d23-43c1-b9bc-1ea33898fab2,El Dorado,-120.298000,38.774000,572.0,2013-07-08,2013-07-11,2013,3
...,...,...,...,...,...,...,...,...,...,...
229,Apple Fire,c7cb91c5-4423-4965-8f6d-ad9c711a376d,Yuba,-121.304167,39.323167,39.0,2022-07-27,2022-08-02,2022,6
230,Oak Fire,ad465a00-5520-4188-9f91-3c5568adb40e,Placer,-120.952600,39.049410,22.0,2022-08-15,2022-08-31,2022,16
231,Pleasant Fire,8d4e0591-dd62-4c04-82f3-10eb76556cde,Nevada,-121.132900,39.280500,48.0,2022-08-20,2022-08-29,2022,9
232,Still Fire,4612ddda-90b6-4733-a0a5-6bdeb622614b,Nevada,-121.110300,39.054300,44.0,2022-08-27,2022-08-29,2022,2


In [131]:
yearly_fires = clean_central_data.groupby(["Year"])["Fire ID"].count()
average_duration = clean_central_data.groupby(["Year"])["Duration (Days)"].mean()
average_acres = clean_central_data.groupby(["Year"])["Acres Burned"].mean()
median_duration = clean_central_data.groupby(["Year"])["Duration (Days)"].median()
median_acres = clean_central_data.groupby(["Year"])["Acres Burned"].median()

summary_stats = pd.DataFrame({
    "Number of Fires":yearly_fires,
    "Avg. Acres":average_acres,
    "Median Acres":median_acres,
    "Avg. Duration (Days)":average_duration,
    "Median Duration (Days)":median_duration
})

summary_stats

,Number of Fires,Avg. Acres,Median Acres,Avg. Duration (Days),Median Duration (Days)
Year,,,,,
2013,15,2116.000000,96.0,2.533333,1.0
2014,10,9850.700000,84.5,4.100000,1.5
2015,13,5788.538462,104.0,6.692308,4.0
2016,30,161.066667,87.5,2.200000,1.0
2017,51,819.823529,76.0,166.254902,172.0
2018,27,5906.888889,67.0,148.629630,156.0
2019,24,76.541667,50.0,2.958333,1.0
2020,33,1791.424242,56.0,1.909091,1.0
2021,19,3837.789474,118.0,10.578947,2.0


In [135]:
show = clean_central_data.loc[clean_central_data["Year"] == 2017]
show

,Name of Fire,Fire ID,County,Longitude,Latitude,Acres Burned,Date Created,Date Extinguished,Year,Duration (Days)
68,Oroville Spillway,afb8c00c-ef91-4544-90ce-54722be124af,Butte,-121.556330,39.511580,0.0,2017-02-07,2018-01-09,2017,336
69,Lombardi Fire,42f21a81-e0a3-4b4d-b837-2e6e2efd75a4,Calaveras,-120.834840,38.199060,15.0,2017-05-28,2018-01-09,2017,226
70,Twenty Fire,67d31514-620f-49d1-b4bd-a3a92efd48ca,Yuba,-121.376000,39.232050,10.0,2017-05-29,2018-01-09,2017,225
71,Clay Fire,bf3148c5-75dd-4f79-8b3b-c5b483065b47,Sacramento,-121.111130,38.337070,68.0,2017-06-01,2018-01-09,2017,222
72,Wilbur Fire,549222c5-1be6-4453-896a-caebd19b0bee,Butte,-121.648970,39.524990,91.0,2017-06-06,2018-01-09,2017,217
73,Skyway Fire,a2d42b3a-32a4-4b57-a8c2-91e5d046bbb6,Butte,-121.754170,39.697170,134.0,2017-06-16,2018-01-09,2017,207
74,Evans Fire,5882937b-ffce-43af-9e0a-00dcddde214a,Sierra,-120.023250,39.711010,168.0,2017-06-19,2018-01-09,2017,204
75,Highway Fire,981484d0-711c-4a7b-9adc-23c9b29f9fae,Butte,-121.414300,39.364000,21.0,2017-06-22,2018-01-09,2017,201
76,Lone Fire,1b1ba02e-9db3-4224-9d8d-06d5df529f16,Butte,-121.578600,39.384160,111.0,2017-06-22,2018-01-09,2017,201
77,Latrobe Fire,4f0ff7c1-7c96-4f68-87e8-be4466945c2b,Sacramento,-120.921620,38.458220,35.0,2017-06-27,2018-01-09,2017,196
